# Ablation of Router


## Model Definition


In [ ]:
# =========================
# Router Ablation Top Cell (Fixed)
# =========================
import os, math, gc
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import GradientBoostingRegressor

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# ---------- Globals ----------
LOG2PI  = math.log(2*math.pi)
DEVICE  = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.set_default_dtype(torch.float32)

# ---------- Repro ----------
def set_seed(seed: int = 1):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

# ---------- Utils ----------
def rmse_score(y_true, y_pred):
    return float(np.sqrt(mean_squared_error(y_true, y_pred)))

def zscore_fit(y_train):
    my = float(np.mean(y_train))
    sy = float(np.std(y_train) + 1e-8)
    return my, sy

def _topk_mask(w, k=2):
    _, topi = torch.topk(w, k, dim=-1)
    mask = torch.zeros_like(w).scatter_(-1, topi, 1.0)
    w2 = w * mask
    return w2 / (w2.sum(dim=-1, keepdim=True) + 1e-12)

def _topk_mask_smooth(w, k=2, eps=0.05):
    _, topi = torch.topk(w, k, dim=-1)
    mask = torch.zeros_like(w).scatter_(-1, topi, 1.0)
    w_top = w * mask
    w_top = w_top / (w_top.sum(dim=-1, keepdim=True) + 1e-12)
    return (1.0 - eps) * w_top + (eps / k) * mask

# ---------- Dataset Loader (fixed for Energy & Naval) ----------
def load_dataset(name: str, verbose: bool = False):
    """
    返回 (X, y)，均为 np.float64。只做基本清洗：转数值+去掉含 NaN 的行。
    支持：
      'housing','concrete','wine','energy','yacht',
      'kin8nm'(本地CSV), 'msd'(本地txt), 'protein'(本地CSV),
      'power'(本地xlsx), 'naval'(本地txt)
    """
    def _clean_numeric(df: pd.DataFrame) -> pd.DataFrame:
        df = df.apply(pd.to_numeric, errors="coerce")
        df = df.replace([np.inf, -np.inf], np.nan).dropna(axis=0)
        return df

    name = name.lower().strip()

    if name == "yacht":
        df = pd.read_csv(
            "https://archive.ics.uci.edu/ml/machine-learning-databases/00243/yacht_hydrodynamics.data",
            header=None, sep=r"\s+", engine="python", comment="#", skip_blank_lines=True
        )
        if df.shape[1] == 1:
            df = df[0].astype(str).str.strip().str.split(r"\s+", expand=True)
        df = _clean_numeric(df); assert df.shape[1] >= 7
        X = df.iloc[:, :-1].to_numpy(np.float64); y = df.iloc[:, -1].to_numpy(np.float64)
        if verbose: print(f"[Yacht] X.shape={X.shape} y.shape={y.shape} | y∈[{y.min():.3f},{y.max():.3f}]")
        return X, y

    if name == "housing":
        df = pd.read_csv(
            "https://archive.ics.uci.edu/ml/machine-learning-databases/housing/housing.data",
            header=None, sep=r"\s+", engine="python", comment="#", skip_blank_lines=True
        )
        if df.shape[1] == 1:
            df = df[0].astype(str).str.strip().str.split(r"\s+", expand=True)
        df = _clean_numeric(df); assert df.shape[1] >= 14
        X = df.iloc[:, :-1].to_numpy(np.float64); y = df.iloc[:, -1].to_numpy(np.float64)
        if verbose: print(f"[Housing] X.shape={X.shape} y.shape={y.shape} | y∈[{y.min():.3f},{y.max():.3f}]")
        return X, y

    if name == "concrete":
        df = pd.read_excel(
            "https://archive.ics.uci.edu/ml/machine-learning-databases/concrete/compressive/Concrete_Data.xls"
        )
        df = _clean_numeric(df)
        X = df.iloc[:, :-1].to_numpy(np.float64); y = df.iloc[:, -1].to_numpy(np.float64)
        if verbose: print(f"[Concrete] X.shape={X.shape} y.shape={y.shape} | y∈[{y.min():.3f},{y.max():.3f}]")
        return X, y

    if name == "wine":
        df = pd.read_csv(
            "https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv",
            delimiter=";"
        )
        df = _clean_numeric(df)
        X = df.iloc[:, :-1].to_numpy(np.float64); y = df.iloc[:, -1].to_numpy(np.float64)
        if verbose: print(f"[WineRed] X.shape={X.shape} y.shape={y.shape} | y∈[{y.min():.3f},{y.max():.3f}]")
        return X, y

    if name == "energy":
        # 原表包含 8 个特征 + Y1 + Y2；这里选择 Y2 为 y
        df = pd.read_excel(
            "https://archive.ics.uci.edu/ml/machine-learning-databases/00242/ENB2012_data.xlsx"
        )
        df = _clean_numeric(df)
        X = df.iloc[:, :-2].to_numpy(np.float64)  # 丢掉 Y1、Y2
        y = df.iloc[:, -1].to_numpy(np.float64)   # 取 Y2
        if verbose: print(f"[Energy] X.shape={X.shape} y.shape={y.shape} | y∈[{np.min(y):.3f},{np.max(y):.3f}]")
        return X, y

    # ---- 以下均需本地文件 ----
    if name == "protein":
        local = "protein.csv"
        if not os.path.exists(local): raise FileNotFoundError("[protein] 需要本地文件 'protein.csv'")
        df = pd.read_csv(local, sep=None, engine="python")
        df = _clean_numeric(df)
        y = df.iloc[:, 0].to_numpy(np.float64)
        X = df.iloc[:, 1:].to_numpy(np.float64)
        if verbose:
            y_min, y_max = float(np.nanmin(y)), float(np.nanmax(y))
            print(f"[Protein(local)] X.shape={X.shape} y.shape={y.shape} | y∈[{y_min:.3f},{y_max:.3f}]")
        return X, y

    if name == "naval":
        local = "naval.txt"
        if not os.path.exists(local): raise FileNotFoundError("[naval] 需要本地文件 'naval.txt'")
        df = pd.read_csv(local, sep=r"\s+", header=None, engine="python")
        df = _clean_numeric(df)
        # UCI Naval 常见有 2 个目标；这里默认取“最后一列”为 y
        # 如需 Y1，可改成 y = df.iloc[:, -2]
        X = df.iloc[:, :-1].to_numpy(np.float64)
        y = df.iloc[:, -1].to_numpy(np.float64)
        if verbose: print(f"[Naval(local)] X.shape={X.shape} y.shape={y.shape} | y∈[{np.min(y):.6f},{np.max(y):.6f}]")
        return X, y

    if name == "kin8nm":
        local = "kin8nm.csv"
        if not os.path.exists(local): raise FileNotFoundError("[kin8nm] 需要本地文件 'kin8nm.csv'")
        df = pd.read_csv(local); df = _clean_numeric(df)
        X = df.iloc[:, :-1].to_numpy(np.float64); y = df.iloc[:, -1].to_numpy(np.float64)
        if verbose: print(f"[Kin8nm] X.shape={X.shape} y.shape={y.shape} | y∈[{y.min():.3f},{y.max():.3f}]")
        return X, y

    if name == "msd":
        local = "YearPredictionMSD.txt"
        if not os.path.exists(local): raise FileNotFoundError("[msd] 需要本地文件 'YearPredictionMSD.txt'")
        df = pd.read_csv(local, header=None); df = df.iloc[:, ::-1]
        df = _clean_numeric(df)
        X = df.iloc[:, :-1].to_numpy(np.float64); y = df.iloc[:, -1].to_numpy(np.float64)
        if verbose: print(f"[MSD] X.shape={X.shape} y.shape={y.shape} | y∈[{y.min():.3f},{y.max():.3f}]")
        return X, y

    if name == "power":
        local = "power.xlsx"  # UCI CCPP (通常名为 Folds5x2_pp.xlsx)
        if not os.path.exists(local): raise FileNotFoundError("[power] 需要本地文件 'power.xlsx'")
        df = pd.read_excel(local); df = _clean_numeric(df)
        X = df.iloc[:, :-1].to_numpy(np.float64); y = df.iloc[:, -1].to_numpy(np.float64)
        if verbose: print(f"[Power(local)] X.shape={X.shape} y.shape={y.shape} | y∈[{y.min():.3f},{y.max():.3f}]")
        return X, y

    raise ValueError("Unknown dataset.")

# ---------- MoE blocks (Router ablation) ----------
class Projection(nn.Module):
    def __init__(self, d, D):
        super().__init__()
        self.w = nn.Linear(d, D, bias=True)
        nn.init.xavier_uniform_(self.w.weight); nn.init.zeros_(self.w.bias)
    def forward(self, x): return self.w(x)

class Window(nn.Module):
    def __init__(self, K, D, min_log_s=-2.5, max_log_s=1.0):
        super().__init__()
        self.c         = nn.Parameter(torch.randn(K, D))
        self.log_s     = nn.Parameter(torch.zeros(K, D))
        self.min_log_s = min_log_s; self.max_log_s = max_log_s
    def forward(self, z):
        log_s = torch.clamp(self.log_s, min=self.min_log_s, max=self.max_log_s)
        diff2 = ((z[:, None] - self.c)**2) / (2 * torch.exp(log_s)**2)
        return torch.exp(-diff2.sum(dim=-1)) + 1e-12  # [B,K]

class ExpertMDN(nn.Module):
    def __init__(self, d, h, nc, sigma_min=5e-2, sigma_max=1.0, learn_mean=True):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(d, h), nn.ReLU(),
            nn.Linear(h, h), nn.ReLU()
        )
        self.logits = nn.Linear(h, nc)
        self.learn_mean = learn_mean
        if learn_mean:
            self.means  = nn.Linear(h, nc)
        self.log_sc = nn.Linear(h, nc)
        self.sigma_min = float(sigma_min)
        self.sigma_max = float(sigma_max)
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight); nn.init.zeros_(m.bias)
    def forward(self, x):
        h  = self.net(x)
        pi = F.softmax(self.logits(h), dim=-1)
        mu = self.means(h) if self.learn_mean else None
        sg = torch.exp(self.log_sc(h)).clamp(self.sigma_min, self.sigma_max)
        return pi, mu, sg

class BLRMoE_NoRouter(nn.Module):
    """
    Router ablation:
      - 去掉 Router（无 q·k）；gating 仅由 Window(z) 决定
      - 支持 mean_mode: 'anchor' | 'anchor+delta' | 'free'
    """
    def __init__(self, d, D, K, hid, nc,
                 mean_mode='anchor+delta', delta_l2=3e-3,
                 l2_win=1e-4,
                 sigma_min=5e-2, sigma_max=1.0,
                 topk=2, smooth_eps=0.05):
        super().__init__()
        assert mean_mode in ['anchor','anchor+delta','free']
        assert 1 <= topk <= K
        self.mean_mode = mean_mode
        self.delta_l2  = float(delta_l2)
        self.proj   = Projection(d, D)
        self.win    = Window(K, D)
        learn_mean = (mean_mode != 'anchor')
        self.exps   = nn.ModuleList([ExpertMDN(d, hid, nc, sigma_min, sigma_max, learn_mean=learn_mean) for _ in range(K)])
        self.l2_win     = l2_win
        self.topk       = topk
        self.smooth_eps = smooth_eps
        self.K = K; self.nc = nc

    def _gating(self, z, train=True):
        w = self.win(z)  # [B,K] >= 0
        w = w / (w.sum(dim=-1, keepdim=True) + 1e-12)
        w = _topk_mask_smooth(w, k=self.topk, eps=self.smooth_eps) if train else _topk_mask(w, k=self.topk)
        return w

    def _mixture_params(self, X, train=True):
        z = self.proj(X)
        w = self._gating(z, train=train)

        B, K, C = X.size(0), self.K, self.nc
        Pi = torch.full((B, K, C), 1.0/C, device=X.device)
        Mu = torch.zeros(B, K, C, device=X.device)
        Sg = torch.ones(B, K, C, device=X.device)

        _, topi = torch.topk(w, self.topk, dim=-1)
        uniq = torch.unique(topi)
        for j in uniq.tolist():
            pi_j, mu_j, sg_j = self.exps[j](X)
            mask = (topi == j).any(dim=1).float().unsqueeze(-1)
            Pi[:, j, :] = pi_j * mask + Pi[:, j, :]*(1 - mask)
            if mu_j is not None:
                Mu[:, j, :] = mu_j * mask + Mu[:, j, :]*(1 - mask)
            Sg[:, j, :] = sg_j * mask + Sg[:, j, :]*(1 - mask)
        return w, Pi, Mu, Sg, z

    def nll(self, X, y_z, mu_anchor_z=None, epoch=1, warmup_epochs=150):
        w, Pi, Mu, Sg, z = self._mixture_params(X, train=self.training)

        if self.mean_mode == 'anchor':
            assert mu_anchor_z is not None
            Mu_eff = mu_anchor_z[:, None, None]
            delta_pen = 0.0
        elif self.mean_mode == 'anchor+delta':
            assert mu_anchor_z is not None
            Mu_eff = mu_anchor_z[:, None, None] + Mu
            delta_pen = (Mu**2).mean() * self.delta_l2
        else:
            Mu_eff = Mu
            delta_pen = 0.0

        yv = y_z[:, None, None]
        logp = -0.5 * ((yv - Mu_eff)/Sg)**2 - torch.log(Sg) - 0.5*LOG2PI

        w3   = w[:, :, None]
        logw = torch.where(w3 > 0, torch.log(w3 + 1e-12), torch.full_like(w3, -1e9))
        logpi= torch.log(Pi + 1e-12)

        logmix = torch.logsumexp(logw + logpi + logp, dim=(1,2))
        nll = -logmix.mean()

        if self.training:
            l2w = (self.win.log_s**2).mean()
            nll = nll + self.l2_win*l2w + delta_pen
        return nll

    @torch.no_grad()
    def predict_mean_var(self, X, mu_anchor_z=None):
        w, Pi, Mu, Sg, _ = self._mixture_params(X, train=False)
        if self.mean_mode == 'anchor':
            assert mu_anchor_z is not None
            Mu_eff = mu_anchor_z[:, None, None]
        elif self.mean_mode == 'anchor+delta':
            assert mu_anchor_z is not None
            Mu_eff = mu_anchor_z[:, None, None] + Mu
        else:
            Mu_eff = Mu

        mu_e  = (Pi * Mu_eff).sum(dim=2)              # [B,K]
        m2_e  = (Pi * (Sg**2 + Mu_eff**2)).sum(dim=2) # [B,K]
        mu_z  = (w * mu_e).sum(dim=1)                 # [B]
        second= (w * m2_e).sum(dim=1)                 # [B]
        var_z = torch.clamp(second - mu_z**2, min=1e-9)
        return mu_z, var_z

# ---------- One-split training (No-leak; with Anchor + CAL) ----------
def train_one_split_router_off(
    X_all, y_all, X_te, y_te,
    standardize_x=True,
    D=2, K=8, HID=128, NC=3,
    LR=1e-3, EPOCHS=400,
    MEAN_MODE='anchor+delta',
    DELTA_L2=3e-3,
    SIGMA_MIN=5e-2, SIGMA_MAX=1.0,
    TOPK=2, SMOOTH_EPS=0.05,
    seed=1
):
    """
    无 Router；其余流程同 Anchor-MoE（含 Anchor 耦合特征 & CAL 线性后校准），严格无泄漏。
    """
    # ---- TV/CAL ----
    X_tv, X_cal, y_tv, y_cal = train_test_split(X_all, y_all, test_size=0.125, random_state=seed)
    # ---- TV -> TR/VA ----
    X_tr, X_va, y_tr, y_va   = train_test_split(X_tv,  y_tv,  test_size=0.2,   random_state=seed)

    # 1) Anchor：TR/VA 选 best_it
    gbdt_full_tr = GradientBoostingRegressor(
        n_estimators=2000, learning_rate=0.05, max_depth=3, subsample=1.0, random_state=seed
    ).fit(X_tr, y_tr)
    val_rmse = [rmse_score(y_va, p) for p in gbdt_full_tr.staged_predict(X_va)]
    best_it  = int(np.argmin(val_rmse)) + 1

    gbdt_sub   = GradientBoostingRegressor(n_estimators=best_it, learning_rate=0.05,
                                           max_depth=3, subsample=1.0, random_state=seed).fit(X_tr, y_tr)
    gbdt_final = GradientBoostingRegressor(n_estimators=best_it, learning_rate=0.05,
                                           max_depth=3, subsample=1.0, random_state=seed).fit(X_tv, y_tv)

    # 2) z-score 与 anchor
    my_tr, sy_tr = zscore_fit(y_tr)
    my_tv, sy_tv = zscore_fit(y_tv)

    mu_tr_anchor = (gbdt_sub.predict(X_tr) - my_tr) / sy_tr
    mu_va_anchor = (gbdt_sub.predict(X_va) - my_tr) / sy_tr

    # 3) 耦合 anchor 特征
    X_tr_aug = np.column_stack([X_tr, mu_tr_anchor])
    X_va_aug = np.column_stack([X_va, mu_va_anchor])

    # 4) 标准化（以 TR 为基）
    if standardize_x:
        mx_tr = X_tr_aug.mean(0, keepdims=True); sx_tr = X_tr_aug.std(0, keepdims=True) + 1e-8
    else:
        mx_tr = np.zeros((1, X_tr_aug.shape[1])); sx_tr = np.ones((1, X_tr_aug.shape[1]))

    def to_tensor(x, yz=None, mx=None, sx=None):
        Xt = torch.tensor(((x - mx)/sx).astype(np.float32), device=DEVICE)
        if yz is None: return Xt
        Yt = torch.tensor(yz.astype(np.float32), device=DEVICE)
        return Xt, Yt

    y_tr_z = (y_tr - my_tr) / sy_tr
    y_va_z = (y_va - my_tr) / sy_tr

    model = BLRMoE_NoRouter(
        d=X_tr_aug.shape[1], D=D, K=K, hid=HID, nc=NC,
        mean_mode=MEAN_MODE, delta_l2=DELTA_L2,
        sigma_min=SIGMA_MIN, sigma_max=SIGMA_MAX,
        topk=TOPK, smooth_eps=SMOOTH_EPS
    ).to(DEVICE)
    opt = optim.AdamW(model.parameters(), lr=LR, weight_decay=3e-4)

    Xtr_t, ytr_t = to_tensor(X_tr_aug, y_tr_z, mx_tr, sx_tr)
    Xva_t, yva_t = to_tensor(X_va_aug, y_va_z, mx_tr, sx_tr)
    mu_tr_t = torch.tensor(mu_tr_anchor.astype(np.float32), device=DEVICE)
    mu_va_t = torch.tensor(mu_va_anchor.astype(np.float32), device=DEVICE)

    best_ep, best_vnll, best_state = 0, +1e9, None
    for ep in range(1, EPOCHS+1):
        model.train(); opt.zero_grad()
        loss = model.nll(Xtr_t, ytr_t, mu_anchor_z=mu_tr_t, epoch=ep, warmup_epochs=150)
        loss.backward(); nn.utils.clip_grad_norm_(model.parameters(), 2.0); opt.step()

        model.eval()
        with torch.no_grad():
            vnll = float(model.nll(Xva_t, yva_t, mu_anchor_z=mu_va_t).cpu().item())
        if vnll < best_vnll:
            best_vnll = vnll; best_ep = ep
            best_state = {k: v.detach().cpu().clone() for k, v in model.state_dict().items()}

    # 5) Phase-2：TV 复训；并为 TV/TE/CAL 生成 anchor
    mu_tv_anchor = (gbdt_final.predict(X_tv) - my_tv) / sy_tv
    mu_te_anchor = (gbdt_final.predict(X_te) - my_tv) / sy_tv
    mu_cal_anchor= (gbdt_final.predict(X_cal)- my_tv) / sy_tv

    X_tv_aug  = np.column_stack([X_tv,  mu_tv_anchor])
    X_te_aug  = np.column_stack([X_te,  mu_te_anchor])
    X_cal_aug = np.column_stack([X_cal, mu_cal_anchor])

    if standardize_x:
        mx_tv = X_tv_aug.mean(0, keepdims=True); sx_tv = X_tv_aug.std(0, keepdims=True) + 1e-8
    else:
        mx_tv = np.zeros((1, X_tv_aug.shape[1])); sx_tv = np.ones((1, X_tv_aug.shape[1]))

    y_tv_z = (y_tv - my_tv) / sy_tv
    y_te_z = (y_te - my_tv) / sy_tv

    model.load_state_dict(best_state)
    model.train()
    opt = optim.AdamW(model.parameters(), lr=LR, weight_decay=3e-4)

    Xtv_t, ytv_t = to_tensor(X_tv_aug, y_tv_z, mx_tv, sx_tv)
    Xte_t        = to_tensor(X_te_aug, None,   mx_tv, sx_tv)
    Xcal_t       = to_tensor(X_cal_aug, None,  mx_tv, sx_tv)
    mu_tv_t  = torch.tensor(mu_tv_anchor.astype(np.float32),  device=DEVICE)
    mu_te_t  = torch.tensor(mu_te_anchor.astype(np.float32),  device=DEVICE)
    mu_cal_t = torch.tensor(mu_cal_anchor.astype(np.float32), device=DEVICE)

    for ep in range(1, best_ep+1):
        opt.zero_grad()
        loss = model.nll(Xtv_t, ytv_t, mu_anchor_z=mu_tv_t, epoch=ep, warmup_epochs=150)
        loss.backward(); nn.utils.clip_grad_norm_(model.parameters(), 2.0); opt.step()

    # 6) CAL 线性后校准，TEST 评估
    model.eval()
    with torch.no_grad():
        yte_t = torch.tensor(y_te_z.astype(np.float32), device=DEVICE)
        test_nll = float(model.nll(Xte_t, yte_t, mu_anchor_z=mu_te_t).cpu().item())

        mu_z_cal, _ = model.predict_mean_var(Xcal_t, mu_anchor_z=mu_cal_t)
        mu_cal_orig = mu_z_cal.cpu().numpy().astype(np.float64) * sy_tv + my_tv

        mu_z_te, _  = model.predict_mean_var(Xte_t,  mu_anchor_z=mu_te_t)
        mu_te_orig  = mu_z_te.cpu().numpy().astype(np.float64) * sy_tv + my_tv

    A = np.vstack([mu_cal_orig, np.ones_like(mu_cal_orig)]).T
    a, b = np.linalg.lstsq(A, y_cal.astype(np.float64), rcond=None)[0]
    mu_te_cal  = a * mu_te_orig + b

    rmse = rmse_score(y_te.astype(np.float64), mu_te_cal)
    r2   = float(r2_score(y_te.astype(np.float64), mu_te_cal))
    return rmse, test_nll, best_ep, r2


## Housing


In [ ]:
# 例：Housing，20×(90/10) 外层划分
set_seed(1)
X, y = load_dataset("housing", verbose=True)

n = X.shape[0]
splits = []
rng = np.random.RandomState(1)
for i in range(20):
    perm = rng.permutation(n)
    splits.append((perm[: round(n*0.9)], perm[round(n*0.9):]))

rmses, nlls = [], []
for i, (tr_idx, te_idx) in enumerate(splits, 1):
    rmse, nll, best_ep, r2 = train_one_split_router_off(
        X[tr_idx].copy(), y[tr_idx].copy(),
        X[te_idx].copy(),  y[te_idx].copy(),
        D=2, K=8, HID=128, NC=3,
        LR=1e-3, EPOCHS=400,
        SIGMA_MIN=5e-2, SIGMA_MAX=1.0,
        TOPK=2, SMOOTH_EPS=0.05,
        seed=1+i
    )
    print(f"[{i:02d}/20] best_ep={best_ep:3d}  RMSE={rmse:.4f}  NLL(z)={nll:.4f}  R²={r2:.4f}")
    rmses.append(rmse); nlls.append(nll)

rmses = np.array(rmses); nlls = np.array(nlls)
se = lambda a: a.std(ddof=1)/np.sqrt(len(a))
print("\n== MoE (Router Ablation, with Anchor + CAL, no-leak) on Housing ==")
print(f"RMSE = {rmses.mean():.4f} ± {se(rmses):.4f}")
print(f"NLL  = {nlls.mean():.4f} ± {se(nlls):.4f}")

[Housing] X.shape=(506, 13) y.shape=(506,) | y∈[5.000,50.000]
[01/20] best_ep=  8  RMSE=2.6053  NLL(z)=0.3143  R²=0.8787
[02/20] best_ep= 16  RMSE=2.7610  NLL(z)=0.4358  R²=0.8786
[03/20] best_ep= 14  RMSE=3.4645  NLL(z)=0.7230  R²=0.8544
[04/20] best_ep=  9  RMSE=2.4260  NLL(z)=0.3063  R²=0.9396
[05/20] best_ep= 14  RMSE=3.2127  NLL(z)=0.9847  R²=0.9189
[06/20] best_ep= 11  RMSE=2.6928  NLL(z)=0.4585  R²=0.9108
[07/20] best_ep=  9  RMSE=1.8283  NLL(z)=0.0781  R²=0.8972
[08/20] best_ep=  7  RMSE=3.5190  NLL(z)=0.4713  R²=0.8252
[09/20] best_ep=  9  RMSE=3.3076  NLL(z)=0.7731  R²=0.8687
[10/20] best_ep=  8  RMSE=4.3314  NLL(z)=0.6275  R²=0.7964
[11/20] best_ep=  9  RMSE=3.1307  NLL(z)=0.8264  R²=0.9006
[12/20] best_ep= 11  RMSE=3.2477  NLL(z)=0.1629  R²=0.8459
[13/20] best_ep= 14  RMSE=2.2793  NLL(z)=0.7567  R²=0.9227
[14/20] best_ep=  9  RMSE=2.2978  NLL(z)=0.4888  R²=0.9504
[15/20] best_ep= 10  RMSE=2.7671  NLL(z)=0.5348  R²=0.8992
[16/20] best_ep=  9  RMSE=3.1925  NLL(z)=0.4066  R²=0

## Concrete



In [ ]:
# 例：Housing，20×(90/10) 外层划分
set_seed(1)
X, y = load_dataset("concrete", verbose=True)

n = X.shape[0]
splits = []
rng = np.random.RandomState(1)
for i in range(20):
    perm = rng.permutation(n)
    splits.append((perm[: round(n*0.9)], perm[round(n*0.9):]))

rmses, nlls = [], []
for i, (tr_idx, te_idx) in enumerate(splits, 1):
    rmse, nll, best_ep, r2 = train_one_split_router_off(
        X[tr_idx].copy(), y[tr_idx].copy(),
        X[te_idx].copy(),  y[te_idx].copy(),
        D=2, K=8, HID=128, NC=3,
        LR=1e-3, EPOCHS=400,
        SIGMA_MIN=5e-2, SIGMA_MAX=1.0,
        TOPK=2, SMOOTH_EPS=0.05,
        seed=1+i
    )
    print(f"[{i:02d}/20] best_ep={best_ep:3d}  RMSE={rmse:.4f}  NLL(z)={nll:.4f}  R²={r2:.4f}")
    rmses.append(rmse); nlls.append(nll)

rmses = np.array(rmses); nlls = np.array(nlls)
se = lambda a: a.std(ddof=1)/np.sqrt(len(a))
print("\n== MoE (Router Ablation, with Anchor + CAL, no-leak) on Housing ==")
print(f"RMSE = {rmses.mean():.4f} ± {se(rmses):.4f}")
print(f"NLL  = {nlls.mean():.4f} ± {se(nlls):.4f}")

[Concrete] X.shape=(1030, 8) y.shape=(1030,) | y∈[2.332,82.599]
[01/20] best_ep= 14  RMSE=4.7123  NLL(z)=0.2781  R²=0.9275
[02/20] best_ep= 17  RMSE=4.2180  NLL(z)=0.3678  R²=0.9310
[03/20] best_ep= 13  RMSE=3.7558  NLL(z)=-0.1359  R²=0.9521
[04/20] best_ep= 14  RMSE=4.6757  NLL(z)=0.1788  R²=0.9204
[05/20] best_ep= 16  RMSE=5.1519  NLL(z)=0.5474  R²=0.8892
[06/20] best_ep= 15  RMSE=3.5783  NLL(z)=-0.2381  R²=0.9515
[07/20] best_ep= 17  RMSE=4.7975  NLL(z)=0.4492  R²=0.9219
[08/20] best_ep= 18  RMSE=5.0889  NLL(z)=0.1390  R²=0.9016
[09/20] best_ep= 15  RMSE=4.1946  NLL(z)=-0.0887  R²=0.9396
[10/20] best_ep= 15  RMSE=5.1534  NLL(z)=0.7313  R²=0.9083
[11/20] best_ep= 18  RMSE=3.3168  NLL(z)=-0.0573  R²=0.9546
[12/20] best_ep= 16  RMSE=4.3361  NLL(z)=0.2097  R²=0.9297
[13/20] best_ep= 14  RMSE=4.4016  NLL(z)=0.1952  R²=0.9328
[14/20] best_ep= 16  RMSE=3.5618  NLL(z)=0.1405  R²=0.9490
[15/20] best_ep= 16  RMSE=4.8248  NLL(z)=0.3035  R²=0.8868
[16/20] best_ep= 18  RMSE=5.1922  NLL(z)=0.3494

## Energy

In [ ]:
# 例：Housing，20×(90/10) 外层划分
set_seed(1)
X, y = load_dataset("energy", verbose=True)

n = X.shape[0]
splits = []
rng = np.random.RandomState(1)
for i in range(20):
    perm = rng.permutation(n)
    splits.append((perm[: round(n*0.9)], perm[round(n*0.9):]))

rmses, nlls = [], []
for i, (tr_idx, te_idx) in enumerate(splits, 1):
    rmse, nll, best_ep, r2 = train_one_split_router_off(
        X[tr_idx].copy(), y[tr_idx].copy(),
        X[te_idx].copy(),  y[te_idx].copy(),
        D=2, K=8, HID=128, NC=3,
        LR=1e-3, EPOCHS=400,
        SIGMA_MIN=5e-2, SIGMA_MAX=1.0,
        TOPK=2, SMOOTH_EPS=0.05,
        seed=1+i
    )
    print(f"[{i:02d}/20] best_ep={best_ep:3d}  RMSE={rmse:.4f}  NLL(z)={nll:.4f}  R²={r2:.4f}")
    rmses.append(rmse); nlls.append(nll)

rmses = np.array(rmses); nlls = np.array(nlls)
se = lambda a: a.std(ddof=1)/np.sqrt(len(a))
print("\n== MoE (Router Ablation, with Anchor + CAL, no-leak) on Housing ==")
print(f"RMSE = {rmses.mean():.4f} ± {se(rmses):.4f}")
print(f"NLL  = {nlls.mean():.4f} ± {se(nlls):.4f}")

[Energy] X.shape=(768, 8) y.shape=(768,) | y∈[10.900,48.030]
[01/20] best_ep= 41  RMSE=1.0822  NLL(z)=-0.9052  R²=0.9873
[02/20] best_ep=291  RMSE=1.1954  NLL(z)=-0.8744  R²=0.9861
[03/20] best_ep= 23  RMSE=1.0062  NLL(z)=-0.9035  R²=0.9886
[04/20] best_ep= 40  RMSE=1.4348  NLL(z)=-0.8177  R²=0.9711
[05/20] best_ep= 35  RMSE=1.4789  NLL(z)=-0.5157  R²=0.9768
[06/20] best_ep= 31  RMSE=1.5282  NLL(z)=-0.7360  R²=0.9713
[07/20] best_ep= 90  RMSE=1.2590  NLL(z)=-0.1996  R²=0.9853
[08/20] best_ep= 25  RMSE=1.5734  NLL(z)=-0.7308  R²=0.9751
[09/20] best_ep= 18  RMSE=0.8936  NLL(z)=-0.8905  R²=0.9904
[10/20] best_ep= 23  RMSE=1.3440  NLL(z)=-0.9223  R²=0.9795
[11/20] best_ep= 95  RMSE=1.0525  NLL(z)=-0.5799  R²=0.9881
[12/20] best_ep= 31  RMSE=1.1252  NLL(z)=-1.0880  R²=0.9856
[13/20] best_ep= 19  RMSE=1.4780  NLL(z)=-1.0114  R²=0.9776
[14/20] best_ep=157  RMSE=1.1230  NLL(z)=-0.8295  R²=0.9855
[15/20] best_ep=358  RMSE=1.0410  NLL(z)=-0.8093  R²=0.9880
[16/20] best_ep=106  RMSE=1.1269  NLL(z

## Kin8nm

In [ ]:
# 例：Housing，20×(90/10) 外层划分
set_seed(1)
X, y = load_dataset("kin8nm", verbose=True)

n = X.shape[0]
splits = []
rng = np.random.RandomState(1)
for i in range(20):
    perm = rng.permutation(n)
    splits.append((perm[: round(n*0.9)], perm[round(n*0.9):]))

rmses, nlls = [], []
for i, (tr_idx, te_idx) in enumerate(splits, 1):
    rmse, nll, best_ep, r2 = train_one_split_router_off(
        X[tr_idx].copy(), y[tr_idx].copy(),
        X[te_idx].copy(),  y[te_idx].copy(),
        D=2, K=8, HID=128, NC=3,
        LR=1e-3, EPOCHS=400,
        SIGMA_MIN=5e-2, SIGMA_MAX=1.0,
        TOPK=2, SMOOTH_EPS=0.05,
        seed=1+i
    )
    print(f"[{i:02d}/20] best_ep={best_ep:3d}  RMSE={rmse:.4f}  NLL(z)={nll:.4f}  R²={r2:.4f}")
    rmses.append(rmse); nlls.append(nll)

rmses = np.array(rmses); nlls = np.array(nlls)
se = lambda a: a.std(ddof=1)/np.sqrt(len(a))
print("\n== MoE (Router Ablation, with Anchor + CAL, no-leak) on Housing ==")
print(f"RMSE = {rmses.mean():.4f} ± {se(rmses):.4f}")
print(f"NLL  = {nlls.mean():.4f} ± {se(nlls):.4f}")

[Kin8nm] X.shape=(8192, 8) y.shape=(8192,) | y∈[0.040,1.459]
[01/20] best_ep=  6  RMSE=0.1528  NLL(z)=0.9933  R²=0.6765
[02/20] best_ep=  6  RMSE=0.1556  NLL(z)=1.0332  R²=0.6574
[03/20] best_ep=  6  RMSE=0.1527  NLL(z)=1.1308  R²=0.6509
[04/20] best_ep=  6  RMSE=0.1450  NLL(z)=0.9631  R²=0.6991
[05/20] best_ep=  6  RMSE=0.1509  NLL(z)=1.0290  R²=0.6280
[06/20] best_ep=  7  RMSE=0.1424  NLL(z)=1.0792  R²=0.7224
[07/20] best_ep=  6  RMSE=0.1557  NLL(z)=1.0210  R²=0.6240
[08/20] best_ep=  7  RMSE=0.1397  NLL(z)=0.9290  R²=0.7451
[09/20] best_ep=  6  RMSE=0.1436  NLL(z)=0.9775  R²=0.6977
[10/20] best_ep=  5  RMSE=0.1506  NLL(z)=1.0547  R²=0.6836
[11/20] best_ep=  7  RMSE=0.1393  NLL(z)=0.9625  R²=0.7356
[12/20] best_ep=  7  RMSE=0.1529  NLL(z)=1.0555  R²=0.6600
[13/20] best_ep=  5  RMSE=0.1503  NLL(z)=0.9522  R²=0.6933
[14/20] best_ep=  6  RMSE=0.1456  NLL(z)=0.9856  R²=0.6907
[15/20] best_ep=  6  RMSE=0.1412  NLL(z)=0.8974  R²=0.7195
[16/20] best_ep=  6  RMSE=0.1494  NLL(z)=0.9905  R²=0.

## Naval

In [ ]:
# 例：Housing，20×(90/10) 外层划分
set_seed(1)
X, y = load_dataset("naval", verbose=True)

n = X.shape[0]
splits = []
rng = np.random.RandomState(1)
for i in range(20):
    perm = rng.permutation(n)
    splits.append((perm[: round(n*0.9)], perm[round(n*0.9):]))

rmses, nlls = [], []
for i, (tr_idx, te_idx) in enumerate(splits, 1):
    rmse, nll, best_ep, r2 = train_one_split_router_off(
        X[tr_idx].copy(), y[tr_idx].copy(),
        X[te_idx].copy(),  y[te_idx].copy(),
        D=2, K=8, HID=128, NC=3,
        LR=1e-3, EPOCHS=400,
        SIGMA_MIN=5e-2, SIGMA_MAX=1.0,
        TOPK=2, SMOOTH_EPS=0.05,
        seed=1+i
    )
    print(f"[{i:02d}/20] best_ep={best_ep:3d}  RMSE={rmse:.4f}  NLL(z)={nll:.4f}  R²={r2:.4f}")
    rmses.append(rmse); nlls.append(nll)

rmses = np.array(rmses); nlls = np.array(nlls)
se = lambda a: a.std(ddof=1)/np.sqrt(len(a))
print("\n== MoE (Router Ablation, with Anchor + CAL, no-leak) on Housing ==")
print(f"RMSE = {rmses.mean():.4f} ± {se(rmses):.4f}")
print(f"NLL  = {nlls.mean():.4f} ± {se(nlls):.4f}")

[Naval(local)] X.shape=(11934, 17) y.shape=(11934,) | y∈[0.975000,1.000000]
[01/20] best_ep= 87  RMSE=0.0006  NLL(z)=-1.1981  R²=0.9934
[02/20] best_ep= 82  RMSE=0.0009  NLL(z)=-1.1024  R²=0.9854
[03/20] best_ep= 99  RMSE=0.0007  NLL(z)=-0.9135  R²=0.9922
[04/20] best_ep=212  RMSE=0.0006  NLL(z)=-1.2326  R²=0.9932
[05/20] best_ep=144  RMSE=0.0007  NLL(z)=-0.8919  R²=0.9923
[06/20] best_ep= 64  RMSE=0.0007  NLL(z)=-0.8797  R²=0.9925
[07/20] best_ep=121  RMSE=0.0006  NLL(z)=-1.0320  R²=0.9934
[08/20] best_ep=397  RMSE=0.0006  NLL(z)=-1.1559  R²=0.9946
[09/20] best_ep=101  RMSE=0.0006  NLL(z)=-1.1355  R²=0.9930
[10/20] best_ep=232  RMSE=0.0006  NLL(z)=-0.9679  R²=0.9928
[11/20] best_ep=110  RMSE=0.0006  NLL(z)=-1.0523  R²=0.9935
[12/20] best_ep=293  RMSE=0.0006  NLL(z)=-1.1137  R²=0.9936
[13/20] best_ep=127  RMSE=0.0007  NLL(z)=-1.0909  R²=0.9921
[14/20] best_ep=259  RMSE=0.0005  NLL(z)=-1.2004  R²=0.9948
[15/20] best_ep=151  RMSE=0.0006  NLL(z)=-1.1489  R²=0.9939
[16/20] best_ep=124  RMS

## Power

In [ ]:
set_seed(1)
X, y = load_dataset("power", verbose=True)

n = X.shape[0]
splits = []
rng = np.random.RandomState(1)
for i in range(20):
    perm = rng.permutation(n)
    splits.append((perm[: round(n*0.9)], perm[round(n*0.9):]))

rmses, nlls = [], []
for i, (tr_idx, te_idx) in enumerate(splits, 1):
    rmse, nll, best_ep, r2 = train_one_split_router_off(
        X[tr_idx].copy(), y[tr_idx].copy(),
        X[te_idx].copy(),  y[te_idx].copy(),
        D=2, K=8, HID=128, NC=3,
        LR=1e-3, EPOCHS=400,
        SIGMA_MIN=5e-2, SIGMA_MAX=1.0,
        TOPK=2, SMOOTH_EPS=0.05,
        seed=1+i
    )
    print(f"[{i:02d}/20] best_ep={best_ep:3d}  RMSE={rmse:.4f}  NLL(z)={nll:.4f}  R²={r2:.4f}")
    rmses.append(rmse); nlls.append(nll)

rmses = np.array(rmses); nlls = np.array(nlls)
se = lambda a: a.std(ddof=1)/np.sqrt(len(a))
print("\n== MoE (Router Ablation, with Anchor + CAL, no-leak) on Housing ==")
print(f"RMSE = {rmses.mean():.4f} ± {se(rmses):.4f}")
print(f"NLL  = {nlls.mean():.4f} ± {se(nlls):.4f}")

[Power(local)] X.shape=(9568, 4) y.shape=(9568,) | y∈[420.260,495.760]
[01/20] best_ep= 25  RMSE=3.2342  NLL(z)=-0.1510  R²=0.9624
[02/20] best_ep= 57  RMSE=3.5139  NLL(z)=-0.0653  R²=0.9584
[03/20] best_ep= 20  RMSE=3.0139  NLL(z)=-0.2232  R²=0.9699
[04/20] best_ep= 22  RMSE=3.0128  NLL(z)=-0.2590  R²=0.9668
[05/20] best_ep= 19  RMSE=3.6344  NLL(z)=-0.1241  R²=0.9561
[06/20] best_ep= 55  RMSE=3.5673  NLL(z)=-0.0333  R²=0.9565
[07/20] best_ep= 25  RMSE=3.0617  NLL(z)=-0.1754  R²=0.9671
[08/20] best_ep= 53  RMSE=3.0818  NLL(z)=-0.1687  R²=0.9679
[09/20] best_ep= 19  RMSE=3.4341  NLL(z)=-0.1461  R²=0.9604
[10/20] best_ep= 95  RMSE=3.3008  NLL(z)=0.0172  R²=0.9618
[11/20] best_ep= 21  RMSE=3.0169  NLL(z)=-0.1340  R²=0.9681
[12/20] best_ep= 22  RMSE=3.3524  NLL(z)=-0.0448  R²=0.9623
[13/20] best_ep= 45  RMSE=2.9765  NLL(z)=-0.2535  R²=0.9703
[14/20] best_ep= 48  RMSE=2.9554  NLL(z)=-0.1962  R²=0.9697
[15/20] best_ep= 23  RMSE=3.3003  NLL(z)=-0.2281  R²=0.9610
[16/20] best_ep= 18  RMSE=2.90

## Protein

In [ ]:
# === Run on Protein (10k downsample) — Router Ablation (with Anchor + CAL, no-leak) ===
set_seed(1)

# 1) 加载数据
X, y = load_dataset("protein", verbose=True)

# 2) 降采样到 10,000（先做降采样，再做切分，避免任何信息泄漏）
MAX_N = 10_000
rng_ds = np.random.RandomState(1)  # 降采样固定种子以可复现
n_full = X.shape[0]
if n_full > MAX_N:
    keep_idx = rng_ds.choice(n_full, size=MAX_N, replace=False)
    X = X[keep_idx].copy()
    y = y[keep_idx].copy()
print(f"[Protein] downsampled: {n_full} -> {X.shape[0]}")

# 3) 20 次 90/10 外层划分
n = X.shape[0]
rng = np.random.RandomState(1)
splits = []
for i in range(20):
    perm = rng.permutation(n)
    splits.append((perm[: round(n * 0.9)], perm[round(n * 0.9):]))

# 4) 训练与评估（Router 关闭版本）
STANDARDIZE_X = True
D, K, HID, NC = 2, 8, 128, 3
LR, EPOCHS    = 1e-3, 400
SIGMA_MIN, SIGMA_MAX = 5e-2, 1.0
TOPK, SMOOTH_EPS = 2, 0.05

rmses, nlls = [], []
for i, (tr_idx, te_idx) in enumerate(splits, 1):
    rmse, nll, best_ep, r2 = train_one_split_router_off(
        X[tr_idx].copy(), y[tr_idx].copy(),
        X[te_idx].copy(),  y[te_idx].copy(),
        D=D, K=K, HID=HID, NC=NC,
        LR=LR, EPOCHS=EPOCHS,
        SIGMA_MIN=SIGMA_MIN, SIGMA_MAX=SIGMA_MAX,
        TOPK=TOPK, SMOOTH_EPS=SMOOTH_EPS,
        seed=1 + i
    )
    print(f"[{i:02d}/20] best_ep={best_ep:3d}  RMSE={rmse:.4f}  NLL(z)={nll:.4f}  R²={r2:.4f}")
    rmses.append(rmse); nlls.append(nll)
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

# 5) 汇总
rmses = np.array(rmses); nlls = np.array(nlls)
se = lambda a: a.std(ddof=1) / np.sqrt(len(a))
print("\n== MoE (Router Ablation, with Anchor + CAL, no-leak) on Protein (10k) ==")
print(f"RMSE = {rmses.mean():.4f} ± {se(rmses):.4f}")
print(f"NLL  = {nlls.mean():.4f} ± {se(nlls):.4f}")

[Protein(local)] X.shape=(45730, 9) y.shape=(45730,) | y∈[0.000,20.999]
[Protein] downsampled: 45730 -> 10000
[01/20] best_ep=  6  RMSE=4.2650  NLL(z)=1.0274  R²=0.5230
[02/20] best_ep=  7  RMSE=4.4836  NLL(z)=1.1656  R²=0.4544
[03/20] best_ep=  6  RMSE=4.3765  NLL(z)=1.1102  R²=0.5026
[04/20] best_ep= 87  RMSE=4.3210  NLL(z)=0.8869  R²=0.5160
[05/20] best_ep=  6  RMSE=4.4092  NLL(z)=1.0418  R²=0.4870
[06/20] best_ep= 50  RMSE=4.4048  NLL(z)=0.9372  R²=0.4621
[07/20] best_ep=  5  RMSE=4.5262  NLL(z)=1.1422  R²=0.4557
[08/20] best_ep=  4  RMSE=4.4356  NLL(z)=1.1459  R²=0.4610
[09/20] best_ep=  5  RMSE=4.2557  NLL(z)=1.0701  R²=0.4922
[10/20] best_ep=  5  RMSE=4.3820  NLL(z)=1.0539  R²=0.5069
[11/20] best_ep=  3  RMSE=4.3189  NLL(z)=1.0704  R²=0.4725
[12/20] best_ep=  5  RMSE=4.4878  NLL(z)=1.1241  R²=0.4558
[13/20] best_ep= 70  RMSE=4.4511  NLL(z)=1.0055  R²=0.4690
[14/20] best_ep=  6  RMSE=4.3288  NLL(z)=1.1723  R²=0.5186
[15/20] best_ep=129  RMSE=4.4782  NLL(z)=0.7762  R²=0.4731
[16/2

## Wine

In [13]:
set_seed(1)
X, y = load_dataset("wine", verbose=True)

n = X.shape[0]
splits = []
rng = np.random.RandomState(1)
for i in range(20):
    perm = rng.permutation(n)
    splits.append((perm[: round(n*0.9)], perm[round(n*0.9):]))

rmses, nlls = [], []
for i, (tr_idx, te_idx) in enumerate(splits, 1):
    rmse, nll, best_ep, r2 = train_one_split_router_off(
        X[tr_idx].copy(), y[tr_idx].copy(),
        X[te_idx].copy(),  y[te_idx].copy(),
        D=2, K=8, HID=128, NC=3,
        LR=1e-3, EPOCHS=400,
        SIGMA_MIN=5e-2, SIGMA_MAX=1.0,
        TOPK=2, SMOOTH_EPS=0.05,
        seed=1+i
    )
    print(f"[{i:02d}/20] best_ep={best_ep:3d}  RMSE={rmse:.4f}  NLL(z)={nll:.4f}  R²={r2:.4f}")
    rmses.append(rmse); nlls.append(nll)

rmses = np.array(rmses); nlls = np.array(nlls)
se = lambda a: a.std(ddof=1)/np.sqrt(len(a))
print("\n== MoE (Router Ablation, with Anchor + CAL, no-leak) on Housing ==")
print(f"RMSE = {rmses.mean():.4f} ± {se(rmses):.4f}")
print(f"NLL  = {nlls.mean():.4f} ± {se(nlls):.4f}")

[WineRed] X.shape=(1599, 11) y.shape=(1599,) | y∈[3.000,8.000]
[01/20] best_ep=  3  RMSE=0.6191  NLL(z)=1.1486  R²=0.4779
[02/20] best_ep=  4  RMSE=0.5933  NLL(z)=1.1272  R²=0.4309
[03/20] best_ep=  3  RMSE=0.7340  NLL(z)=1.4439  R²=0.3934
[04/20] best_ep=  2  RMSE=0.5916  NLL(z)=1.1027  R²=0.4483
[05/20] best_ep=  4  RMSE=0.5465  NLL(z)=1.0120  R²=0.4642
[06/20] best_ep=  4  RMSE=0.6048  NLL(z)=1.1110  R²=0.4296
[07/20] best_ep=  2  RMSE=0.6740  NLL(z)=1.2361  R²=0.4062
[08/20] best_ep=  3  RMSE=0.6553  NLL(z)=1.2097  R²=0.4387
[09/20] best_ep=  2  RMSE=0.5934  NLL(z)=1.0664  R²=0.4281
[10/20] best_ep=  3  RMSE=0.6312  NLL(z)=1.2219  R²=0.4748
[11/20] best_ep=  4  RMSE=0.6203  NLL(z)=1.2297  R²=0.4588
[12/20] best_ep= 20  RMSE=0.6549  NLL(z)=1.1588  R²=0.2508
[13/20] best_ep=  3  RMSE=0.6458  NLL(z)=1.2793  R²=0.3456
[14/20] best_ep=  4  RMSE=0.6289  NLL(z)=1.1836  R²=0.2503
[15/20] best_ep=  5  RMSE=0.5853  NLL(z)=1.0740  R²=0.5255
[16/20] best_ep= 11  RMSE=0.5634  NLL(z)=1.0104  R²=

## Yacht

In [14]:
set_seed(1)
X, y = load_dataset("yacht", verbose=True)

n = X.shape[0]
splits = []
rng = np.random.RandomState(1)
for i in range(20):
    perm = rng.permutation(n)
    splits.append((perm[: round(n*0.9)], perm[round(n*0.9):]))

rmses, nlls = [], []
for i, (tr_idx, te_idx) in enumerate(splits, 1):
    rmse, nll, best_ep, r2 = train_one_split_router_off(
        X[tr_idx].copy(), y[tr_idx].copy(),
        X[te_idx].copy(),  y[te_idx].copy(),
        D=2, K=8, HID=128, NC=3,
        LR=1e-3, EPOCHS=400,
        SIGMA_MIN=5e-2, SIGMA_MAX=1.0,
        TOPK=2, SMOOTH_EPS=0.05,
        seed=1+i
    )
    print(f"[{i:02d}/20] best_ep={best_ep:3d}  RMSE={rmse:.4f}  NLL(z)={nll:.4f}  R²={r2:.4f}")
    rmses.append(rmse); nlls.append(nll)

rmses = np.array(rmses); nlls = np.array(nlls)
se = lambda a: a.std(ddof=1)/np.sqrt(len(a))
print("\n== MoE (Router Ablation, with Anchor + CAL, no-leak) on Housing ==")
print(f"RMSE = {rmses.mean():.4f} ± {se(rmses):.4f}")
print(f"NLL  = {nlls.mean():.4f} ± {se(nlls):.4f}")

[Yacht] X.shape=(308, 6) y.shape=(308,) | y∈[0.010,62.420]
[01/20] best_ep=283  RMSE=0.2997  NLL(z)=-1.9975  R²=0.9996
[02/20] best_ep=137  RMSE=0.5804  NLL(z)=-1.7256  R²=0.9983
[03/20] best_ep=289  RMSE=0.4620  NLL(z)=-1.9131  R²=0.9983
[04/20] best_ep=170  RMSE=0.7143  NLL(z)=-1.7618  R²=0.9984
[05/20] best_ep=362  RMSE=0.5734  NLL(z)=-1.8615  R²=0.9989
[06/20] best_ep=249  RMSE=0.5007  NLL(z)=-1.8520  R²=0.9981
[07/20] best_ep=370  RMSE=0.2688  NLL(z)=-2.0164  R²=0.9982
[08/20] best_ep=345  RMSE=0.7860  NLL(z)=-1.5648  R²=0.9971
[09/20] best_ep=137  RMSE=0.5190  NLL(z)=-1.8564  R²=0.9984
[10/20] best_ep=390  RMSE=0.4961  NLL(z)=-1.7786  R²=0.9972
[11/20] best_ep=391  RMSE=0.3863  NLL(z)=-1.9419  R²=0.9995
[12/20] best_ep=246  RMSE=0.7766  NLL(z)=-1.6490  R²=0.9960
[13/20] best_ep=122  RMSE=0.9339  NLL(z)=-1.5199  R²=0.9967
[14/20] best_ep=248  RMSE=0.8091  NLL(z)=-1.5349  R²=0.9975
[15/20] best_ep=289  RMSE=0.6055  NLL(z)=-1.7349  R²=0.9985
[16/20] best_ep= 47  RMSE=0.7278  NLL(z)=